# Run Test Method

In [8]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Modelo

In [9]:
from __future__ import annotations
import ast, sys
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Optional

@dataclass(frozen=True)
class CodeWarning:
    message: str
    lineno: int
    col: int
    end_lineno: Optional[int] = None
    end_col: Optional[int] = None

class DetectionRule:
    name: str = "Generic Detector"
    def search(self, src: str) -> List[CodeWarning]:
        raise NotImplementedError

class Patcher:
    name: str = "Generic Patcher"
    def patch(self, tree: ast.Node) -> str:
        raise NotImplementedError

class DCCLinter:
    def __init__(self, detectors: Iterable[DetectionRule], patchers: Iterable[Patcher]):
        self.detectors = detectors
        self.patchers = patchers

    def search(self, src: str) -> List[CodeWarning]:
        out: List[CodeWarning] = []
        for d in self.detectors:
            out.extend(d.search(src))
        return sorted(out, key=lambda w: (w.lineno, w.col))

    def patch(self, src: str) -> str:
        tree = ast.parse(src)
        for p in self.patchers:
          tree = p.patch(tree)
        return ast.unparse(tree)

# None Compares

## Detector

In [10]:
class SubprocessDetectionRule(DetectionRule):
    name = "Detect subprocess.call/check_call o os.system"

    def search(self, src: str) -> List[CodeWarning]:
        # Se ajustaron patrones a subprocess/os.
        tree = ast.parse(src)
        warns: List[CodeWarning] = []
        for node in ast.walk(tree):
            if isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute):
                base = node.func.value
                if isinstance(base, ast.Name) and base.id == 'subprocess' and node.func.attr in ('call','check_call'):
                    warns.append(CodeWarning(f"Use subprocess.run en lugar de subprocess.{node.func.attr}", node.lineno, node.col_offset))
                if isinstance(base, ast.Name) and base.id == 'os' and node.func.attr == 'system':
                    warns.append(CodeWarning("Use subprocess.run en lugar de os.system", node.lineno, node.col_offset))
        return warns

## Patcher

In [11]:
class SubprocessPatcher(Patcher):
    name = "Fix to subprocess.run([...], check=True)"

    def patch(self, tree: ast.AST) -> ast.AST:
        # Reescribe call/check_call/os.system a run(..., check=True).
        for node in ast.walk(tree):
            if isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute):
                if node.func.attr in ('call','check_call'):
                    node.func.attr = 'run'
                    node.keywords = [ast.keyword(arg='check', value=ast.Constant(True))]
                elif node.func.attr == 'system' and isinstance(node.func.value, ast.Name) and node.func.value.id == 'os':
                    node.func.value.id = 'subprocess'
                    node.func.attr = 'run'
                    node.keywords = [
                        ast.keyword(arg='shell', value=ast.Constant(True)),
                        ast.keyword(arg='check', value=ast.Constant(True))
                    ]
        return tree

## Tests

In [12]:
import unittest

class TestSubprocessModernize(unittest.TestCase):
    def setUp(self):
        self.l = DCCLinter([SubprocessDetectionRule()], [SubprocessPatcher()])

    def test_detect(self):
        s = "import subprocess\nsubprocess.call(['echo','hi'])\n"
        ws = self.l.search(s)
        self.assertEqual(len(ws), 1)

    def test_patch_call(self):
        s = "import subprocess\nsubprocess.call(['echo','hi'])\n"
        out = self.l.patch(s)
        self.assertIn("subprocess.run(['echo', 'hi'], check=True)", out)

    def test_patch_os_system(self):
        s = "import os, subprocess\nos.system('echo hi')\n"
        out = self.l.patch(s)
        self.assertIn("subprocess.run('echo hi', shell=True, check=True)", out)

    # Tests Extra
    def test_patch_check_call(self):
        s = "import subprocess\nsubprocess.check_call(['echo','hi'])\n"
        out = self.l.patch(s)
        self.assertIn("subprocess.run(['echo', 'hi'], check=True)", out)

    def test_no_warning_for_run(self):
        s = "import subprocess\nsubprocess.run(['echo','hi'])\n"
        ws = self.l.search(s)
        self.assertEqual(len(ws), 0)

    def test_detect_multiple(self):
        s = (
            "import os, subprocess\n"
            "subprocess.call(['echo','a'])\n"
            "os.system('echo b')\n"
        )
        ws = self.l.search(s)
        self.assertEqual(len(ws), 2)

In [13]:
runTests(TestSubprocessModernize)

......
----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK
.....
----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


## Real World Code

In [14]:
def find_cell(comment):
    """
    Busca la primera celda en el cuaderno actual cuya primera línea
    comience con el comentario dado (por ejemplo, "# Case Study 1").

    Parámetros:
        comment (str): El comentario que se quiere buscar.

    Retorna:
        str | None: El código fuente de la celda encontrada, o None si no existe.
    """
    for cell_source in In:  # `In` contiene el código fuente de todas las celdas ejecutadas
        lines = cell_source.strip().splitlines()
        if lines and lines[0].strip().startswith(comment):  # revisa la primera línea
            return cell_source
    return None

In [15]:
# Case Study 1
import os, subprocess
subprocess.check_call(['ls','-la'])
os.system('echo hi')


# Otra llamada heredada
subprocess.call(['echo','bye'])

# Dentro de un if
if True:
    os.system('echo inside-if')

# Dentro de una función
def run_legacy():
    subprocess.check_call(['echo','from-function'])


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [ ]:
linter = DCCLinter([SubprocessDetectionRule()],[SubprocessPatcher()])

bad_code =  find_cell("# Case Study 1")
warnings = linter.search(bad_code)
print("-= Warnings =-")
for w in warnings:
    print(w)
good_code = linter.patch(bad_code)
print("-= Fixed Code =-")
print(good_code)

-= Warnings =-
CodeWarning(message='Use subprocess.run en lugar de subprocess.check_call', lineno=3, col=0, end_lineno=None, end_col=None)
CodeWarning(message='Use subprocess.run en lugar de os.system', lineno=4, col=0, end_lineno=None, end_col=None)
-= Fixed Code =-
import os, subprocess
subprocess.run(['ls', '-la'], check=True)
subprocess.run('echo hi', shell=True, check=True)
